In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Datensätze Laden

In [3]:
alt_ges = pd.read_csv('./data/alter_geschlecht.csv', sep=',')
rest = pd.read_csv('./data/rest.csv', sep=';')
interesse = pd.read_csv("./data/interesse.csv", sep=',')

# Vorläufige Datenanalyse

In [8]:
print("Alter-Geschlecht:", alt_ges.shape)
print("Rest:", rest.shape)
print("Interesse:", interesse.shape)

Alter-Geschlecht: (508146, 3)
Rest: (508146, 9)
Interesse: (381109, 2)


### Duplizierte IDs

In [ ]:
print("Alter-Geschlecht:", alt_ges['id'].duplicated().sum())
print("Rest:", rest['id'].duplicated().sum())
print("Interesse:", interesse['id'].duplicated().sum())

Alter-Geschlecht: 0
Rest: 0
Interesse: 0


### Fehlwerte

Es gibt in keinem Datensatz fehlende Werte

In [21]:
print("Alter-Geschlecht:\n", alt_ges.isnull().mean(), "\n\n")
print("Rest:\n", rest.isnull().mean(), '\n\n')
print("Interesse:\n", interesse.isnull().mean())

Alter-Geschlecht:
 id            0.0
Geschlecht    0.0
Alter         0.0
dtype: float64 


Rest:
 Fahrerlaubnis      0.0
Regional_Code      0.0
Vorversicherung    0.0
Alter_Fzg          0.0
Vorschaden         0.0
Jahresbeitrag      0.0
Vertriebskanal     0.0
Kundentreue        0.0
id                 0.0
dtype: float64 


Interesse:
 id           0.0
Interesse    0.0
dtype: float64


# Zusammenführung von Datensätzen

Da die Anzahl der eindeutigen IDs in den verschiedenen Datensätzen unterschiedlich ist, wird die Schnittmenge der IDs in den verschiedenen Datensätzen verwendet.

In [37]:
df = pd.merge(interesse, alt_ges, on='id', how='inner').merge(rest, on='id', how='inner').sort_values(by='id').set_index('id')
df['Interesse'] = df['Interesse'].astype(int)

In [31]:
df.shape

(381109, 11)

In [30]:
df

,Interesse,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue
id,,,,,,,,,,,
1,1.0,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217
2,0.0,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183
3,1.0,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27
4,0.0,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203
5,0.0,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39
...,...,...,...,...,...,...,...,...,...,...,...
381105,0.0,Male,74,1,26.0,1,1-2 Year,No,30170.0,26.0,88
381106,0.0,Male,30,1,37.0,1,< 1 Year,No,40016.0,152.0,131
381107,0.0,Male,21,1,30.0,1,< 1 Year,No,35118.0,160.0,161


In [38]:
df.dtypes

Interesse            int64
Geschlecht          object
Alter                int64
Fahrerlaubnis        int64
Regional_Code      float64
Vorversicherung      int64
Alter_Fzg           object
Vorschaden          object
Jahresbeitrag      float64
Vertriebskanal     float64
Kundentreue          int64
dtype: object

In [41]:
df['Interesse'].describe()

count    381109.000000
mean          0.122563
std           0.327936
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Interesse, dtype: float64